# Assignment 1

#### Student ID: M124111043

#### Name: 唐嘉宏

In [1]:
%pip install --pre pycaret[full] -qq
%pip install cleanlab -qq
%pip install snorkel -qq


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Q1: Exploring a Low-Code ML Framework with Melbourne Housing dataset

After completing the California census data project, we will shift our focus to a different housing dataset.

The dataset is a snapshot of a [dataset](https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market) created by Tony Pino. This dataset is a compilation of housing market data for Melbourne, which was sourced from weekly public listings on [Domain.com.au](https://www.domain.com.au/). It includes various details such as the address, type of real estate, suburb, selling method, number of rooms, price, real estate agent, date of sale, and the distance from the central business district, among others. **Ensure to set the random seed to 2024 for reproducibility.**

To begin, execute the following code snippet for data preparation:


In [1]:
import numpy as np
import pandas as pd
from pycaret.regression import *

# Prepare data
df = pd.read_csv("melb_data.csv")
df = df.drop(['Date', 'Address'], axis=1)
df = df.copy()
df

,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,2,h,1480000.0,S,Biggin,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,2,h,1035000.0,S,Biggin,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,3,h,1465000.0,SP,Biggin,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,3,h,850000.0,PI,Biggin,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,4,h,1600000.0,VB,Nelson,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,4,h,1245000.0,S,Barry,16.7,3150.0,4.0,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,3,h,1031000.0,SP,Williams,6.8,3016.0,3.0,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,3,h,1170000.0,S,Raine,6.8,3016.0,3.0,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,4,h,2500000.0,PI,Sweeney,6.8,3016.0,4.0,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


**(a) Data Splitting**: Using the `train_test_split()` function, divide the dataset into two parts: a training set and a test set. Allocate 5% of the data to the test set, which will be used for testing purposes. (5%)

In [12]:
# coding your answer here.
from sklearn.model_selection import train_test_split

X = df.drop('Price', axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=2024)

print(f"training set: {len(X_train)}")
print(f"testing set: {len(X_test)}")

training set: 12901
testing set: 679


**(b) Dataset Preparation Using PyCaret:**
Prepare the dataset for our machine learning model using the `setup()` function from `PyCaret` with the following requirements: (10%)

- **Features**: Include all features in the `df` dataframe, except for the `Price` column, which will be used as the target.
- **Target**: Set `Price` as the target variable.
- **Validation Set**: Split 20% of the data into a validation set to evaluate the model.
- **Missing Values**:
  - For numerical variables, replace missing values with the median value of the respective columns. Assume that columns of data type `object` are categorical variables, and all others are numerical.
  - For categorical variables, replace missing values with the most frequent value in each column.
- **Encoding**:
  - Apply one-hot encoding to categorical variables that have fewer than 10 categories.
  - Use target encoding for categorical columns with 10 or more categories.
- **Standardization**: All features should be standardized. Note that PyCaret automatically scales all features by default.

**Hint:** Refer to the PyCaret documentation [here](https://pycaret.readthedocs.io/en/latest/api/regression.html) for additional details on setup parameters. Be aware of how PyCaret handles feature scaling by default as discussed [here](https://github.com/pycaret/pycaret/issues/3076).

In [32]:
from pycaret.regression import setup

regression_setup = setup(
    data=df, 
    target='Price', 
    train_size=0.8, 
    numeric_imputation='median', 
    categorical_imputation='mode', 
    normalize=True, 
    normalize_method='zscore', 
    fold=5,
    session_id=2024
)


,Description,Value
0,Session id,2024
1,Target,Price
2,Target type,Regression
3,Original data shape,"(13580, 21)"
4,Transformed data shape,"(13580, 34)"
5,Transformed train set shape,"(10864, 34)"
6,Transformed test set shape,"(2716, 34)"
7,Numeric features,12
8,Categorical features,8
9,Rows with missing values,54.4%


After setting up your dataset with `PyCaret`, you can review the configuration and steps of the created machine learning pipeline by executing the following code snippet. This will help you verify that all preprocessing steps are correctly applied as per the requirements:

In [33]:
# Get the PyCaret configuration
config = get_config('pipeline')
config

Pipeline(memory=FastMemory(location=C:\Users\User\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Rooms', 'Distance', 'Postcode',
                                             'Bedroom2', 'Bathroom', 'Car',
                                             'Landsize', 'BuildingArea',
                                             'YearBuilt', 'Lattitude',
                                             'Longtitude', 'Propertycount'],
                                    transformer=SimpleImputer(strategy='median'))),
                ('categorical_imputer',
                 Transform...
                                    transformer=OneHotEncoder(cols=['Type',
                                                                    'Method',
                                                                    'Regionname'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('rest_encoding',
                 TransformerWrapper(include=['Suburb', 'Address', 'SellerG',
                                             'Date', 'CouncilArea'],
                                    transformer=TargetEncoder(cols=['Suburb',
                                                                    'Address',
                                                                    'SellerG',
                                                                    'Date',
                                                                    'CouncilArea'],
                                                              handle_missing='return_nan'))),
                ('normalize',
                 TransformerWrapper(transformer=StandardScaler()))])

**(c) Verify Transformed Data:**
Use the `get_config()` function in `PyCaret` to inspect the transformed dataset. After executing this function, review the output to ensure the number of samples and the number of features match your expectations based on the data preparation steps previously outlined. (10%)

Provide a brief explanation of why the numbers match or differ from your expectations.

In [34]:
# coding your answer here.
from pycaret.regression import get_config

X_transformed = get_config('X')  
y_transformed = get_config('y')  

print(f"Transformed Features Shape: {X_transformed.shape}")
print(f"Transformed Target Shape: {y_transformed.shape}")


Transformed Features Shape: (13580, 20)
Transformed Target Shape: (13580,)


> Ans: *double click here to answer the question.* <bd>

Column沒有符合預期，應該會是34而不是20, 因為One-hot增加少量列別，但Target壓縮多個類別。
而Row是符合預期的，因為對於缺失值是進行補值處理而非直接刪除。

**(d) Model Comparison and Evaluation:** (5%)

Utilize the `compare_models()` function in `PyCaret` to conduct a 3-fold cross-validation analysis. Compare the performance of the following machine learning models:

- Linear Regression
- Lasso Regression
- Ridge Regression
- Support Vector Regression (SVR)
- K-Nearest Neighbor (KNN) Regressor
- Random Forest Regressor
- XGBoost
- LightGBM
- CatBoost

List the top five models according to their performance on the Mean Absolute Percentage Error (MAPE) measure.

In [37]:
# coding your answer here.
from pycaret.regression import compare_models
top_models = compare_models(
    include=['lr', 'lasso', 'ridge', 'svm', 'knn', 'rf', 'xgboost', 'lightgbm', 'catboost'], 
    fold=3, 
    sort='MAPE'
)
print(top_models)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,241780.1719,177782925994.6666,421262.5312,0.5624,0.2817,0.2135,0.1333
svm,Support Vector Regression,439888.9027,434664069814.0724,659170.1475,-0.0715,0.5281,0.4413,1.5000
lightgbm,Light Gradient Boosting Machine,452351.2961,393559400100.3706,627255.8798,0.0296,0.5363,0.5340,0.2133
xgboost,Extreme Gradient Boosting,452697.9062,392450845354.6667,626416.2708,0.0319,0.5364,0.5350,0.2300
catboost,CatBoost Regressor,452847.6867,393778638276.8960,627408.6826,0.0293,0.5369,0.5352,1.4467
ridge,Ridge Regression,457943.1405,401123234022.8182,633193.9157,0.0114,0.5422,0.5420,0.0733
lasso,Lasso Regression,458094.1555,401348427595.0462,633371.6242,0.0109,0.5424,0.5422,0.0767
lr,Linear Regression,458314.6982,401742915191.1018,633675.7188,0.0100,0.5426,0.5424,0.0800
rf,Random Forest Regressor,459630.9261,404695800943.6272,636063.5634,0.0023,0.5442,0.5434,0.8067


KNeighborsRegressor(n_jobs=-1)


> Ans: *double click here to answer the question.*
1. KNN
2. SVM
3. LightGBM
4. XGBOOST
5. CATBOOST


**(e) Final Model Selection and Evaluation**: Using the `create_model()` function in `PyCaret`, select the best-performing model from part (d) based on the MAPE metric. Evaluate this model on the test set that was reserved in part (a). Report the MAPE on the test set to assess the model’s performance. (5%)

In [40]:
from pycaret.regression import create_model, predict_model, get_config
from sklearn.metrics import mean_absolute_percentage_error

best_model = create_model('knn')

X_test = get_config('X_test')  
y_test = get_config('y_test') 

predictions = predict_model(best_model, data=X_test)

y_pred = predictions['prediction_label']  


mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE on the test set: {mape}")


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,239394.6094,192827703296.0000,439121.5000,0.5384,0.2791,0.2086
1,248183.3750,198395150336.0000,445415.7188,0.5428,0.2841,0.2106
2,240322.4375,161187004416.0000,401481.0000,0.5745,0.2862,0.2192
3,235667.3750,145497948160.0000,381441.9375,0.5905,0.2791,0.2162
4,240605.7031,191070437376.0000,437116.0312,0.5681,0.2733,0.2044
Mean,240834.7000,177795648716.8000,420915.2375,0.5628,0.2804,0.2118
Std,4076.5914,20714780580.7598,25020.1451,0.0196,0.0045,0.0053


MAPE on the test set: 0.20626136660575867


> Ans: *double click here to answer the question.* <bd>

MAPE = 0.2063

## Q2: Handling an Unlabeled Dataset in a Startup Environment

In this exercise, you'll hone your data preparation skills, a crucial element in many real-world projects. You are provided with an unlabeled dataset `data.csv`, which consists of two features and encompasses 300 sample points. Each sample point is associated with one of four distinct classes. Additionally, you have `crowdsourcing.csv` containing labels from 50 different workers for these sample points.

**While this is a practice project, and you have access to the actual ground truth labels in `labels.csv`, these labels are restricted to measuring the accuracy of your models. Under no circumstances should the ground truth labels be used in other parts of the training process.** Ensure to set the random seed to 2024 for all operations requiring randomness to maintain consistency in your results.

Firstly, read the dataset and divide it into training and testing sets using the following code snippet:

In [54]:
import pandas as pd
import numpy as np

X = pd.read_csv('data.csv', names=[0,1])
true_labels = np.loadtxt("labels.csv", delimiter=',')
multiannotator_labels = pd.read_csv('crowdsourcing.csv')

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, multiannotator_labels, ano_unseen, Y_train, Y_test  = train_test_split(X, multiannotator_labels, true_labels, test_size=0.1, random_state=2024)

**(a) Label Aggregation Using Snorkel:** (10%)

In this task, we'll explore the use of `Snorkel` to manage and refine crowdsourced labels. Each crowd worker will be treated as an individual labeling function since each one labels different subsets of the dataset and might introduce errors or conflicts.

**Steps:**

1. **Convert Crowdsourcing Data**: Transform the crowdsourcing data frame into a label matrix where each row corresponds to a sample and each column to a worker's label for that sample. This matrix will be used to train the `Snorkel` model.

2. **Train the Label Model**: Use the `Snorkel` `LabelModel` to train on the label matrix you created. Apply the following hyperparameters:
   ```python
   fit(L_train, n_epochs=500, seed=2024, log_freq=20, l2=0.1, lr=0.001, lr_scheduler="linear", optimizer="adam")
   ```

3. **Generate Predictions and Evaluate Accuracy**: Once trained, use the `LabelModel` to generate predictions for each training sample. Then, calculate the accuracy of these predictions by comparing them against the ground truth labels from `labels.csv`. Remember, the ground truth is only used for evaluating accuracy, not for training the Label Model.

**Hints:**
- You can check the Snorkel documentation [here](https://snorkel.readthedocs.io/en/master/packages/_autosummary/labeling/snorkel.labeling.model.label_model.LabelModel.html) for more details on the `LabelModel` parameters and methods.

In [2]:
from snorkel.labeling.model import LabelModel
from sklearn.metrics import accuracy_score


label_matrix = multiannotator_labels.values

label_model = LabelModel(cardinality=4, verbose=True)

label_model.fit(L_train=label_matrix, 
                n_epochs=500, 
                seed=2024, 
                log_freq=20, 
                l2=0.1, 
                lr=0.001, 
                lr_scheduler="linear", 
                optimizer="adam")

predicted_labels = label_model.predict(L=label_matrix)

accuracy = accuracy_score(Y_train, predicted_labels)
print(f"Accuracy: {accuracy}")

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.167]
INFO:root:[20 epochs]: TRAIN:[loss=0.097]
INFO:root:[40 epochs]: TRAIN:[loss=0.089]
INFO:root:[60 epochs]: TRAIN:[loss=0.089]
 12%|█▏        | 62/500 [00:00<00:00, 604.70epoch/s]INFO:root:[80 epochs]: TRAIN:[loss=0.090]
INFO:root:[100 epochs]: TRAIN:[loss=0.091]
INFO:root:[120 epochs]: TRAIN:[loss=0.091]
 25%|██▌       | 125/500 [00:00<00:00, 613.96epoch/s]INFO:root:[140 epochs]: TRAIN:[loss=0.091]
INFO:root:[160 epochs]: TRAIN:[loss=0.091]
INFO:root:[180 epochs]: TRAIN:[loss=0.091]
 37%|███▋      | 187/500 [00:00<00:00, 607.01epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.091]
INFO:root:[220 epochs]: TRAIN:[loss=0.091]
INFO:root:[240 epochs]: TRAIN:[loss=0.091]
 50%|█████     | 251/500 [00:00<00:00, 618.55epoch/s]INFO:root:[260 epochs]: TRAIN:[loss=0.091]
INFO:root:[280 epochs]: TRAIN:[loss=0.091]
INFO:root:[300 epochs]: TRAIN:[loss=0.091]
 63%|██████▎

Accuracy: 0.8518518518518519


**(b) Refining Labels with Cleanlab:** (10%)

In this second trial, we'll leverage the `cleanlab` library to further analyze and refine the labels obtained from crowdsourcing. The process will involve deriving consensus labels from the crowdsourced data, using majority voting and model predictions.

**Steps:**

1. **Majority Vote Labels**:
   - Utilize the `get_majority_vote_label()` function to aggregate the labels from different workers for each sample into a single majority vote label.
   
2. **Model Predictions**:
   - Apply a 5-fold cross-validation on the training dataset labeled with the majority vote results. Use the model below to compute out-of-sample predicted probabilities for each sample.
   ```python
    LogisticRegression(solver='liblinear', penalty='l1', C=0.1, random_state=2024)
   ```

3. **Consensus Labels**:
   - Employ the `get_label_quality_multiannotator()` function from `cleanlab` to combine the out-of-sample predicted probabilities with the crowdsourced labels. This function will help generate consensus labels, which aim to be more accurate by considering both worker disagreement and model confidence.

4. **Evaluate Accuracy**:
   - Calculate the accuracy of both the majority vote labels and the consensus labels by comparing them against the ground truth labels in `labels.csv`. As with previous tasks, remember that the ground truth is strictly for evaluation purposes.

**Hints:**
- Explore the Cleanlab documentation [here](https://github.com/cleanlab/cleanlab) for additional insights into functions like `get_majority_vote_label()` and `get_label_quality_multiannotator()`.

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cleanlab.classification import CleanLearning


def get_majority_vote_labels(multiannotator_labels):
    from scipy import stats
    majority_vote_labels = stats.mode(multiannotator_labels, axis=1, nan_policy='omit').mode
    return majority_vote_labels.flatten()

majority_vote_labels = get_majority_vote_labels(multiannotator_labels)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train.values)

valid_indices = ~np.isnan(majority_vote_labels)
X_filtered = X_scaled[valid_indices]
y_majority_vote_filtered = majority_vote_labels[valid_indices].astype(int)

logreg = LogisticRegression(solver='liblinear', penalty='l1', C=0.1, random_state=2024)

from sklearn.model_selection import cross_val_predict
predicted_probs = cross_val_predict(logreg, X_filtered, y_majority_vote_filtered, cv=5, method='predict_proba')

clean_model = CleanLearning(clf=logreg)

clean_model.fit(X_filtered, y_majority_vote_filtered)
consensus_labels = clean_model.predict(X_filtered)

consensus_label_accuracy = accuracy_score(Y_train[:len(consensus_labels)], consensus_labels)
majority_vote_accuracy = accuracy_score(Y_train[:len(consensus_labels)], y_majority_vote_filtered)

print(f"Majority Vote Accuracy: {majority_vote_accuracy}")
print(f"Consensus Label Accuracy: {consensus_label_accuracy}")


Majority Vote Accuracy: 0.8481481481481481
Consensus Label Accuracy: 0.6888888888888889


**(c) Evaluate Random Forest Classifiers:** (10%)

Since we don't have these crowdsourcing labels for the test set or new incoming data points, we can't use the `LabelModel` or obtain consensus label at inference time. In order to run inference on new incoming data points, we need to train a discriminative model! Train a model using the following setup:

```python
RandomForestClassifier(random_state=2024, n_estimators=10)
```
**Steps:**
- Train a classifier for each type of label.
    - **Snorkel Labels**: Generated in part (a).
    - **Majority-Vote Labels**: Obtained in part (b) from `get_majority_vote_label()`.
    - **Consensus Labels**: Derived in part (b) from `get_label_quality_multiannotator()`.
- Evaluate and report the classification accuracy on the test set.
- Determine which classifier performs best based on the accuracy scores.

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

snorkel_predicted_labels = label_model.predict(L=multiannotator_labels.values)

def get_majority_vote_labels(multiannotator_labels):
    from scipy import stats
    majority_vote_labels = stats.mode(multiannotator_labels, axis=1, nan_policy='omit').mode
    return majority_vote_labels.flatten()

majority_vote_labels = get_majority_vote_labels(multiannotator_labels)

label_difference = np.sum(snorkel_predicted_labels != majority_vote_labels)
print(f"Difference between Snorkel and Majority Vote labels: {label_difference} samples differ.")

def train_and_evaluate_random_forest(X_train, y_train, X_test, y_test):
    rf_model = RandomForestClassifier(random_state=2024, n_estimators=10)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

snorkel_accuracy = train_and_evaluate_random_forest(X_train, snorkel_predicted_labels, X_test, Y_test)

majority_vote_accuracy = train_and_evaluate_random_forest(X_train, majority_vote_labels, X_test, Y_test)

consensus_label_accuracy = train_and_evaluate_random_forest(X_train, consensus_labels, X_test, Y_test)

# 打印準確度
print(f"Snorkel Labels Accuracy: {snorkel_accuracy}")
print(f"Majority Vote Labels Accuracy: {majority_vote_accuracy}")
print(f"Consensus Labels Accuracy: {consensus_label_accuracy}")


Difference between Snorkel and Majority Vote labels: 25 samples differ.
Snorkel Labels Accuracy: 0.7666666666666667
Majority Vote Labels Accuracy: 0.7666666666666667
Consensus Labels Accuracy: 0.6


> Ans: *double click here to answer the question.* <bd>

Best:
Snokel and Majority vote 
accuracy = 0.7667

**(d) Enhanced Random Forest Training:** (10%)

Train two additional Random Forest classifiers with the specifications:
```python
RandomForestClassifier(random_state=2024, n_estimators=10)
```

**Steps:**
- Fit each classifier using the respective sample weights for the labels.
    - **Snorkel Labels**: Use `inv_entropy()` to calculate weights based on prediction probabilities from the `Snorkel` `LabelModel`.
    - **Consensus Labels**: Use `consensus_quality_score` from `Cleanlab` for weights.
- Evaluate and report the classification accuracy on the test set.
- Compare the results to those from (c) to assess performance improvement.

Hint: Refer to [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.fit) for the `sample_weight` paprameter.

In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from scipy.stats import entropy  # 正確導入 entropy 函數

X = pd.read_csv('data.csv', names=[0,1])
true_labels = np.loadtxt('labels.csv', delimiter=',')
multiannotator_labels = pd.read_csv('crowdsourcing.csv')

X_train, X_test, multiannotator_labels, ano_unseen, Y_train, Y_test = train_test_split(
    X, multiannotator_labels, true_labels, test_size=0.1, random_state=2024
)

snorkel_predicted_labels = label_model.predict(L=multiannotator_labels.values)

def get_majority_vote_labels(multiannotator_labels):
    from scipy import stats
    majority_vote_labels = stats.mode(multiannotator_labels, axis=1, nan_policy='omit').mode
    return majority_vote_labels.flatten()

majority_vote_labels = get_majority_vote_labels(multiannotator_labels)


def inv_entropy(probs):
    """計算反向熵作為樣本權重"""
    return 1.0 / (entropy(probs, axis=1) + 1e-6)  # 避免除以0

snorkel_probs = label_model.predict_proba(L=multiannotator_labels.values)
snorkel_weights = inv_entropy(snorkel_probs)

def train_and_evaluate_random_forest_with_weights(X_train, y_train, X_test, y_test, sample_weights):
    rf_model = RandomForestClassifier(random_state=2024, n_estimators=10)
    rf_model.fit(X_train, y_train, sample_weight=sample_weights)
    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

snorkel_weighted_accuracy = train_and_evaluate_random_forest_with_weights(X_train, snorkel_predicted_labels, X_test, Y_test, snorkel_weights)

consensus_probs = clean_model.predict_proba(X_train)
consensus_weights = inv_entropy(consensus_probs)

consensus_weighted_accuracy = train_and_evaluate_random_forest_with_weights(X_train, consensus_labels, X_test, Y_test, consensus_weights)

print(f"Snorkel Weighted Labels Accuracy: {snorkel_weighted_accuracy}")
print(f"Consensus Weighted Labels Accuracy: {consensus_weighted_accuracy}")


Snorkel Weighted Labels Accuracy: 0.8
Consensus Weighted Labels Accuracy: 0.6


> Ans: *double click here to answer the question.*

在Snornel中從76.67% 提升到 80% 說明應用樣本權重可以提升性能

而Consensus中則是維持60%

## Q3: Analyze StackOverflow dataset using SQL

Kaggle has a rich number of [BigQuery](https://www.kaggle.com/datasets?fileType=bigQuery) and [SQLite](https://www.kaggle.com/datasets?fileType=sqlite) datasets that you can practice your SQL skill.

In this question, we are going to examine the StackOverflow dataset. [Stack Overflow](https://stackoverflow.com/) is a popular question and answer site for technical questions.

Hint: It is recommended to answer this question in Kaggle, where you can access the dataset directly.

Firstly, if you are using colab, use the following code snippet to setup the client. For more detail, please refer to our lab.

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import sys

# Upload JSON file here
from google.colab import files
uploaded = files.upload()

path_of_json = 'nomadic-botany-435608-b6-e1a2d53f264a.json'  #@param {type: "string"}

# TODO(developer): Set path_of_json to the path to the service account key file.

credentials = service_account.Credentials.from_service_account_file(
    path_of_json
)

In [ ]:
if "google.colab" in sys.modules:
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
else:
    # Below and create a "Client" object if you are using Kaggle
    client = bigquery.Client()

**(a) Initial Exploration of the Stack Overflow Dataset:** (10%)

Begin your analysis by familiarizing yourself with the structure and contents of the Stack Overflow dataset:

1. **List All Tables**: Print the names of all tables available in the dataset.
2. **View Table Schemas**: Display the schema for the `posts_questions` and `posts_answers` tables.
3. **Preview Data**: Retrieve and preview the first ten rows from each of the tables mentioned above.

**Hint:** Use the following code snippet to reference the dataset in your client setup:
```python
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
```

In [ ]:
# coding your answer here.
from google.cloud import bigquery

dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))

print("Tables in dataset:")
for table in tables:
    print(table.table_id)

In [ ]:
questions_table_ref = dataset_ref.table("posts_questions")
questions_table = client.get_table(questions_table_ref)

print("\nSchema of posts_questions table:")
for field in questions_table.schema:
    print(f"{field.name}: {field.field_type}")

answers_table_ref = dataset_ref.table("posts_answers")
answers_table = client.get_table(answers_table_ref)

print("\nSchema of posts_answers table:")
for field in answers_table.schema:
    print(f"{field.name}: {field.field_type}")



In [ ]:
query_questions = """
SELECT *
FROM `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 10
"""

questions_query_job = client.query(query_questions)
questions_results = questions_query_job.result()

print("\nFirst 10 rows from posts_questions table:")
for row in questions_results:
    print(dict(row))

**(b) Daily Post Counts for 2020:** (10%)

Write an SQL query to count the number of posts created each day in the `posts_questions` table for the year 2020:

- **Filter by Year**: Use `WHERE` and `EXTRACT()` to select posts from 2020 only.
- **Date Column**: Format the `creation_date` in DATE format (e.g. 2021-01-01) and rename it to "created_day". (Remember the SELECT... from clause, DATE() function and `AS` command).
- **Count Posts**: Count the daily posts and label this column as "number_of_posts". (Remember the `COUNT()` function).
- **Group and Sort**: Group the results by "created_day" using `GROUP BY` and order them by "created_day" in ascending order using `ORDER BY`.

Your query should return a dataframe with 366 rows and two columns, showing the date and corresponding post count.

<center>

| created_day | number_of_posts |
|:-----------:|:---------------:|
|  2020-01-01 |       2390      |
|  2020-01-02 |       4601      |
|  2020-01-03 |       4816      |
|     ...     |       ....   |
|  2020-12-31 |       3498      |

</center>

Generate a line plot with "created_day" on the x-axis and "number_of_posts" on the y-axis to visualize daily posting trends. What patterns do you observe from the line plot regarding post frequency throughout 2020?

In [ ]:
# coding your answer here.
query = """
SELECT
  DATE(creation_date) AS created_day,
  COUNT(*) AS number_of_posts
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
WHERE
  EXTRACT(YEAR FROM creation_date) = 2020
GROUP BY
  created_day
ORDER BY
  created_day ASC
     """

############   Do not modify the code below     ############
# Set up the query (cancel the query if it would use too much of
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
############   Do not modify the code above     ############

In [ ]:
# coding your answer here.
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(results['created_day'], results['number_of_posts'], marker='o')
plt.title('Daily Post Counts on Stack Overflow (2020)', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Posts', fontsize=12)
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

> Ans: *double click here to answer the question.* <bd>

反映發帖數量每天的波動很大, 可能是工作日和周末的影響, 還有某些日期突然增多也許是有特殊事件或技術討論重大發展. 

**(c) Analyzing User Engagement with 'pandas' Tag**: (10%)

`posts_questions` has a column called tags which lists the topics/technologies each question is about. `posts_answers` has a column called `parent_id` which identifies the ID of the question each answer corresponds to. You can then join two tables by the `parent_id` in `posts_answers` and the `id` in `posts_questions`. `posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

Now, write a query that has a single row for each user who answered at least one question with a tag that equals to string `pandas`. Your results should have two columns (Remember the `SELET...WHERE` clause) and follow the below restrictions:

* `owner_user_id` - contains the `owner_user_id` column from the `posts_answers` table (Remember the `INNER JOIN` clause)
* `number_of_answers` - contains the number of answers the user has written to `pandas` related questions using the `tage` column (Remember the `AS`, `GROUP BY` and `COUNT` clause)
* Discard the rows whose `number_of_answers` is smaller or equal to 3 (Remember the `HAVING` clause)

<center>

| owner_user_id | number_of_answers |
|---------------|-------------------|
| 1234567       | 794               |
| 2345678       | 304               |
|     ...     |       ....   |
|  9999999 |       4      |

</center>

In the retrieve data frame, which user answers most questions in the pandas domain? Try to find the email of the user.

Hint: You can find the public user profile by appending the ID after https://stackoverflow.com/users/

In [ ]:
# coding your answer here.
query = """
    SELECT
        a.owner_user_id AS owner_user_id,
        COUNT(a.id) AS number_of_answers
    FROM
        `bigquery-public-data.stackoverflow.posts_answers` a
    INNER JOIN
        `bigquery-public-data.stackoverflow.posts_questions` q
    ON
        a.parent_id = q.id
    WHERE
        q.tags LIKE '%<pandas>%'
    GROUP BY
        owner_user_id
    HAVING
        COUNT(a.id) > 3
    ORDER BY
        number_of_answers DESC
"""
############   Do not modify the code below     ############
# Set up the query (cancel the query if it would use too much of
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
############   Do not modify the code above     ############

In [ ]:
# coding your answer here.

top_user = results.iloc[0]
top_user_id = top_user['owner_user_id']
print(f"The user who answered the most 'pandas' questions has the ID: {top_user_id}")

> Ans: *double click here to answer the question.*

https://stackoverflow.com/users/16343464/mozway